# The plan
1. Create new features based on original table
2. Group by sender and recipient to find where users are trading abnormally high between each other  
3. Check if users send back and forth similar amounts frequently (wash trading)  
4. Compute median time difference between trades for recurring transactions    

### 1. New features from what we have already

In [1]:
# MODULES
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import sys
import os
import sqlite3
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# SCRIPTS
sys.path.append(os.path.abspath(".."))
from scripts import conversions

In [2]:
df_original = pd.read_csv('../data/large/full_transactions.csv')
df_original['timestamp'] = pd.to_datetime(df_original['timestamp'])

In [3]:
# very high ratios could be suspicious
df_original['value_gas_ratio'] = df_original['value'] / df_original['gas_price'] 
df_original['value_gas_ratio'].describe()

count    1.000465e+06
mean     6.953910e-01
std      6.369054e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      5.336583e-03
max      3.473669e+04
Name: value_gas_ratio, dtype: float64

In [4]:
df_original

,hash,sender,recipient,value,gas_used,gas_price,timestamp,value_zscore,value_gas_ratio
0,0x06dc5006e9cc13418808a35b5a6fb91378eb78c236db...,0x675bB023e268dCC43F543620577bCacB73047f08,0x5dC62cea20B0E7c3607aDCc61a885Ff9369dbc60,0.000000,223837,180.411926,2025-02-14 10:16:10,-0.011788,0.000000
1,0x689d1453646e621d664836655d869d7301b8c7bcd1d4...,0x1ba98272731f1Ab5cC33a02F88694C39916E26dF,0x51C72848c68a965f66FA7a88855F9f7784502a7F,0.000000,153985,81.237520,2025-02-14 10:16:10,-0.011788,0.000000
2,0x3057779542c82c2ae3f1b52b6739afe82719a61ad26c...,0x5d1783ddbb657fdf6FcC2eF5B6796Ac0DdeEb6e3,0x68d3A973E7272EB388022a5C6518d9b2a2e66fBf,0.000000,223550,65.091946,2025-02-14 10:16:11,-0.011788,0.000000
3,0x0c664ea5c27efb322c1dc20c2a830b97a33fbd82b027...,0x3f22f60936F4f5D1E96cEdEb44beE66d0Cd7c220,0x98C3d3183C4b8A650614ad179A1a98be0a8d6B8E,0.000000,500002,63.063619,2025-02-14 10:16:11,-0.011788,0.000000
4,0x1b386bb2f08f57a5287640e09e2f3640798a58d69a3a...,0x24f7eF98522dd61D529464f67BB3ffe96eA8AFC2,0x5dC62cea20B0E7c3607aDCc61a885Ff9369dbc60,0.000000,252997,47.995919,2025-02-14 10:16:11,-0.011788,0.000000
...,...,...,...,...,...,...,...,...,...
1000460,0x8cbf01dc0bbebe854a64ccd7e9ccee8876c95ee30cfb...,0x5698D4ec03802438Bce268b02eb3817B8C3C2b3f,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0.000000,45758,1.919426,2025-02-17 06:06:35,-0.011788,0.000000
1000461,0xb0fac9da7e21836663a05e8bcfd429a64e66160f10d7...,0x5baBE600b9fCD5fB7b66c0611bF4896D967b23A1,0xf45AFF520a8Ebe59f513719fc78b164e375Dd1f4,0.007562,21000,1.917395,2025-02-17 06:06:35,-0.011718,0.003944
1000462,0x488d4e454120b061f3a0c41008b90d1156737845d91a...,0x99719eacc100BD48D4dbEb96E990333a7601e5B8,0x65c55F33F9493E8702FEea89B4fA357283eD3841,0.000000,302687,1.917395,2025-02-17 06:06:35,-0.011788,0.000000
1000463,0xcdda18972b9baf4310ce4be72d1ec91dc32bb2f02c6b...,0x6a0913BA185848c09250D3F40DFf29dC45530978,0xb965602E41Af7A92B3F4b48729736Aded9C94E47,0.000000,500000,1.913384,2025-02-17 06:06:35,-0.011788,0.000000


### 2. Frequent transactions between users with very high amounts

In [5]:
conn = sqlite3.connect(':memory:')
df_original.to_sql('transactions', conn, index=False)

query = ''' 
SELECT sender, recipient, value
FROM transactions 
WHERE value_zscore > 2.5
'''
df_high_transactions = pd.read_sql_query(query, conn)
conn.close()

df_high_transactions

,sender,recipient,value
0,0xA7EFAe728D2936e78BDA97dc267687568dD593f3,0x0b2fDF416Cf2951499dE9A1aDaC65c8e9907C8C2,499.440000
1,0xDBF5E9c5206d0dB70a90108bf936DA60221dC080,0x6dEcD32f5A2AB9B43bC6ad8923Eb6D2395dE145b,3627.000000
2,0xEae7380dD4CeF6fbD1144F49E4D1e6964258A4F4,0x28C6c06298d514Db089934071355E5743bf21d60,3626.998570
3,0x0b2fDF416Cf2951499dE9A1aDaC65c8e9907C8C2,0x28C6c06298d514Db089934071355E5743bf21d60,499.759816
4,0xfe9CB5423f54C8d69917c5bEeb8c00503B4a1A91,0x28C6c06298d514Db089934071355E5743bf21d60,681.964490
...,...,...,...
512,0xb5d85CBf7cB3EE0D56b3bB207D5Fc4B82f43F511,0xA9D1e08C7793af67e9d92fe308d5697FB81d3E43,2809.072638
513,0x21a31Ee1afC51d94C2eFcCAa2092aD1028285549,0xDBF5E9c5206d0dB70a90108bf936DA60221dC080,2659.000000
514,0x652a2aDe712e21B9F83672bDE4462c6F8723A30b,0xf7858Da8a6617f7C6d0fF2bcAFDb6D2eeDF64840,799.778527
515,0x21a31Ee1afC51d94C2eFcCAa2092aD1028285549,0x652a2aDe712e21B9F83672bDE4462c6F8723A30b,399.889300


This was just a quick check to see how many transacitons are considered high value, but we generally don't need to modify our original `df` since it should be able to detect these outliers on its own via random partitions

### 3. Transaction frequency between pairs of users in short amounts of time

In [6]:
''' 
Explanation of the query:

- The JOIN will detect when users have a history of sending transactions back and forth
as well as having a time range between one day

- The group by will group all of our aggregates (count and sums) by each tuple of sender and recipient
- The having clause ensures theres a reasonable number of transactions in this time range to be
considered suspicious and that the difference between the back and forth transactions is a similar enough amount
to be considered wash trading

'''
conn = sqlite3.connect(':memory:')
df_original.to_sql('transactions', conn, index=False)


query = ''' 
SELECT 
t1.sender AS sender_addr,
t1.recipient AS recipient_addr,
COUNT(*) AS total_transactions,
SUM(t1.value) AS total_sent,
SUM(t2.value) AS total_received

FROM transactions t1
JOIN transactions t2
ON t1.sender = t2.recipient
AND t1.recipient = t2.sender
AND julianday(t2.timestamp) BETWEEN julianday(t1.timestamp) AND julianday(t1.timestamp) + 1

GROUP BY sender_addr, recipient_addr
HAVING total_transactions > 3
AND ABS(total_sent - total_received) < .01 * total_sent

ORDER BY total_transactions DESC
'''

df_wash_trading = pd.read_sql_query(query, conn)
conn.close()

df_wash_trading


,sender_addr,recipient_addr,total_transactions,total_sent,total_received
0,0x6d42aAb8B4EFd1AD0C5454a040B59Ae3c99440B5,0x6d42aAb8B4EFd1AD0C5454a040B59Ae3c99440B5,46,1380.460000,1380.460000
1,0x00001b0bcca9BC88774000692DEf9Dd6452afb9F,0x00001b0bcca9BC88774000692DEf9Dd6452afb9F,42,58.277158,58.275502
2,0x01cc0bDC248a2916EFB0e39F28aAd2ae3D21be09,0xB74A3fbB5B8BF6607f236c53FB5eE4d2ed536b51,35,0.083212,0.082989
3,0x004c1Bf2F642249a683B850FcE63dDC1c94e836c,0x3c10d28469b918e89e0506cf4fc549dc8a889e1C,10,0.000100,0.000100
4,0x01Ff4825C1f8e902A6aB0F2F28195004Da1E86D6,0x809caa6F8b360BeEe8B4340Ff5Af41fE13873495,10,0.000100,0.000100
...,...,...,...,...,...
1302,0xb0b5B1F7048488D2C974b5d438083F47f397e575,0x1f3Bf5652cDb0aeFcf897F3aE97039614C636916,4,0.000040,0.000040
1303,0xc3a7494837748B23eEE68FA4c894a9c37F68003a,0x07570087D0eD0442e66Be444f3043Bcd9C78F01E,4,0.000040,0.000040
1304,0xda5f693487b7e9450380fF7aAEF07EFb9520b0Ad,0xda5f693487b7e9450380fF7aAEF07EFb9520b0Ad,4,0.004000,0.004000
1305,0xf03fCAF811e112467964684c1C6569e89562323f,0x1B52BeAEf0b3a9c5ed9C5319eBffB5673455a5a4,4,0.000040,0.000040


Here we can find when pairs of users have sent many transactions between eachother with very similar amounts, which can be a sign of wash trading.  

[References heuristic 1 from this article](https://www.chainalysis.com/blog/crypto-market-manipulation-wash-trading-pump-and-dump-2025/#:~:text=The%20first%20heuristic%20suggests%20that,trade%20volume%20in%20November%202024.). Rather than using the difference in block, however, I considered the time period of the transactions.


### 4. Find how often users are sending and receiving transactions

In [7]:
''' 
Explanation of the query:

- We group by sender and transaction_hour because we want to show every time a user has made very frequent transactions
- We use the having clause to filter the transactions after the grouping
- We order by avg_value desc 

'''
conn = sqlite3.connect(':memory:')
df_original.to_sql('transactions', conn, index=False)

query = ''' 
SELECT 
sender,
strftime('%Y-%m-%d %H', timestamp) AS transaction_hour,
COUNT(*) AS total_transactions, 
SUM(value) AS total_value

FROM transactions
GROUP BY sender, transaction_hour
HAVING total_transactions > 5
ORDER BY total_value DESC
'''

df_transaction_rate = pd.read_sql_query(query, conn)
conn.close()

df_transaction_rate

,sender,transaction_hour,total_transactions,total_value
0,0x28C6c06298d514Db089934071355E5743bf21d60,2025-02-17 05,231,34798.113190
1,0x28C6c06298d514Db089934071355E5743bf21d60,2025-02-14 11,243,34624.889273
2,0x28C6c06298d514Db089934071355E5743bf21d60,2025-02-17 04,255,34338.945832
3,0x28C6c06298d514Db089934071355E5743bf21d60,2025-02-16 18,120,33111.253746
4,0x9C19B0497997Fe9E75862688a295168070456951,2025-02-16 15,8,26259.513566
...,...,...,...,...
16124,0xff8Ba4D1fC3762f6154cc942CCF30049A2A0cEC6,2025-02-17 02,16,0.000000
16125,0xff8Ba4D1fC3762f6154cc942CCF30049A2A0cEC6,2025-02-17 03,21,0.000000
16126,0xff8Ba4D1fC3762f6154cc942CCF30049A2A0cEC6,2025-02-17 04,10,0.000000
16127,0xff8Ba4D1fC3762f6154cc942CCF30049A2A0cEC6,2025-02-17 05,17,0.000000


This query is incredibly informative as it shows when users have sent many transactions within the same hour. For example, at index 0, we see that the user has sent 231 transactions with the average value of 34798 ETH, which is a whole lot of money.

### Evaluating from here  
If I was to join these three tables, I would probably get a very large dataset with lots of repeating features. Throwing this information into the isolation forest might not perform as well since each table is checking for a different fraud strategy.  

In order to keep the model simple and more generalizable. It may be better to keep all of the data seperate, train three seperate Isolation Forest models, and then run each new transaction to see what comes up. This will be done in the isolation forest notebook.

In [10]:
df_original.to_csv("../data/large/modified_full_transactions.csv", index=False)
df_wash_trading.to_csv("../data/small/wash_trading.csv", index=False)
df_transaction_rate.to_csv("../data/small/transaction_rate.csv", index=False)